# Federated Learning Training Plan: Host Plan & Model

Here we load Plan and Model params created earlier in "Create Plan" notebook, host them to PyGrid, 
and run sample syft.js app that executes them.  

In [1]:
import asyncio
import websockets
import json
import binascii

async def sendWsMessage(data):
    async with websockets.connect(gatewayWsUrl) as websocket:
        await websocket.send(json.dumps(data))
        message = await websocket.recv()
        return json.loads(message)

## Step 5a: Host in PyGrid

**NOTE**: Right now PyGrid supports only 1 version of plan (list of ops or torchscript).
But Plans will converge to one type here https://github.com/OpenMined/PySyft/issues/2994#issuecomment-595333791.

Here we load "ops list" Plan.

In [2]:
# Load files with protobuf created in "Create Plan" notebook.
with open("model_params.pb", "rb") as model_file, open("tp_ops.pb", "rb") as plan_file: 
    serialized_model = model_file.read()
    serialized_ops_plan = plan_file.read()


Follow PyGrid README.md to build `openmined/grid-gateway` image from the latest `dev` branch 
and spin up PyGrid using `docker-compose up`.

In [3]:
# Default gateway address when running locally 
gatewayWsUrl = "ws://127.0.0.1:5000"

This hand-crafted request emulates `pygrid.host_federated_training` from the roadmap.

In [4]:
# These name/version you use in worker
name = "mnist"
version = "1.0.0"

host_request = {
    "type": "federated/host-training",
    "data": {
        "model": binascii.hexlify(serialized_model).decode("utf-8"),
        "plans": {
            "training_plan": binascii.hexlify(serialized_ops_plan).decode("utf-8"),
            # "extra_plan": ...,
        },
        "protocols": {
            # "secure_agg_protocol": ...,
        },
        "averaging_plan": "",
        "client_config": {
            "name": name,  
            "version": version,
            "batch_size": 64,
            "lr": 0.01,
            "max_updates": 200  # custom syft.js option that limits number of training loops per worker
        },
        "server_config": {
            "max_workers": 100,
            "pool_selection": "random",
            "num_cycles": 5,
            "do_not_reuse_workers_until_cycle": 4,
            "cycle_length": 28800,
            "minimum_upload_speed": 0,
            "minimum_download_speed": 0
        }
    }
}

Shoot!

If everything's good, success is returned.
If the name/version already exists in PyGrid, change them above or cleanup PyGrid db by re-creating docker containers. 


In [5]:
response = await sendWsMessage(host_request)
print("Host response:", response)

Host response: {'status': 'success'}


Let's double-check that data is loaded by requesting a cycle.

In [6]:
auth_request = {
    "type": "federated/authenticate",
    "data": {}
}
auth_response = await sendWsMessage(auth_request)
print('Auth response: ', json.dumps(auth_response, indent=2))

cycle_request = {
    "type": "federated/cycle-request",
    "data": {
        "worker_id": auth_response['worker_id'],
        "model": name,
        "version": version,
        "ping": 1,
        "download": 1000,
        "upload": 1000,
    }
}
cycle_response = await sendWsMessage(cycle_request)
print('Cycle response:', json.dumps(cycle_response, indent=2))


Auth response:  {
  "status": "success",
  "worker_id": "0c785fc4-be45-45e7-b269-82ba6781c425"
}
Cycle response: {
  "status": "accepted",
  "request_key": "5be5738875c8b2afb6c3a9f351b20cd9760f4d6c2a3b129dcfbcc2c0a6bcfa39",
  "model": "mnist",
  "plans": {
    "training_plan": 2
  },
  "protocols": {},
  "client_config": {
    "name": "mnist",
    "version": "1.0.0",
    "batch_size": 64,
    "lr": 0.01,
    "max_updates": 200
  },
  "model_id": 1
}


## Step 6a: Train

Start and open "with-grid" example in syft.js project (http://localhost:8080 by default), 
enter model name and version and start FL training.


